In [ ]:
!git clone https://github.com/4evergr8/Thousand2OneColor /kaggle/working/DDColor

In [ ]:
import os
import shutil

def safe_move(src_path, dst_dir):
    os.makedirs(dst_dir, exist_ok=True)
    base_name = os.path.basename(src_path)
    name, ext = os.path.splitext(base_name)
    dst_path = os.path.join(dst_dir, base_name)

    count = 1
    while os.path.exists(dst_path):
        dst_path = os.path.join(dst_dir, f"{name}_{count}{ext}")
        count += 1

    shutil.move(src_path, dst_path)
    print(f'移动：{src_path} -> {dst_path}')

def move_zip_files(source_root, target_root, subfolder):
    for root, _, files in os.walk(source_root):
        if os.path.basename(root) != subfolder:
            continue  # 只处理 train 或 val 文件夹
        for file in files:
            if file.lower().endswith('.zip'):
                src_file = os.path.join(root, file)
                dst_dir = os.path.join(target_root, subfolder)
                safe_move(src_file, dst_dir)

def main():
    source_root = '/kaggle/input'
    target_root = '/kaggle/temp'

    move_zip_files(source_root, target_root, 'train')
    move_zip_files(source_root, target_root, 'val')

    print('全部移动完成。')

if __name__ == '__main__':
    main()


In [ ]:
import os
import urllib.request
import shutil

work_dir = "/kaggle/working/DDColor"
pretrain_dir = os.path.join(work_dir, "pretrain")
tmp_dir = "/tmp"
experiments_input_dir = "/kaggle/input/experiments"
experiments_output_dir = "/kaggle/output/DDColor/experiments"

def copy_experiments():
    if os.path.exists(experiments_output_dir):
        shutil.rmtree(experiments_output_dir)
    shutil.copytree(experiments_input_dir, experiments_output_dir)
    print(f"已复制 {experiments_input_dir} 到 {experiments_output_dir}")

def download_and_move():
    os.makedirs(pretrain_dir, exist_ok=True)
    urls = [
        "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth",
        "https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_224.pth"
    ]

    for url in urls:
        filename = url.split("/")[-1]
        tmp_path = os.path.join(tmp_dir, filename)
        final_path = os.path.join(pretrain_dir, filename)
        if not os.path.exists(final_path):
            print(f"下载 {filename} 到 /tmp...")
            urllib.request.urlretrieve(url, tmp_path)
            shutil.move(tmp_path, final_path)
            print(f"已移动到 {final_path}")
        else:
            print(f"{filename} 已存在，跳过。")

if os.path.exists(experiments_input_dir):
    copy_experiments()
else:
    download_and_move()


In [ ]:
!cd /kaggle/temp && python3 /kaggle/working/DDColor/get_meta_file.py

In [ ]:
import os
base_path = "/kaggle/working/DDColor"
os.chdir(base_path)
train_script = os.path.join(base_path, "basicsr", "train.py")
config_file = os.path.join(base_path, "Kaggle.yml")
command = f'PYTHONPATH="{base_path}:$PYTHONPATH" python3 "{train_script}" -opt "{config_file}" --auto_resume'
!{command}
